In [1]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import torch.nn as nn 
import os
from skimage import io, transform
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np


In [2]:
import torch.nn.functional as F

In [3]:
class ImageNetDataset(Dataset):
    
    def __init__(self, root_dir, data_folder, transform=None):
        self.data_path = os.path.join(root_dir,data_folder)
        self.data = os.listdir(self.data_path)
        self.root = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_name = self.data[index]
        img_path = os.path.join(self.data_path,img_name)
        image = io.imread(img_path)
        if self.transform:
            image = self.transform(image)
        
        return image,img_name

In [4]:
class addGaussian:
    
    def __init__(self, std_range):
        self.std_range = std_range

    def __call__(self,img):
        minval,maxval = self.std_range
        noise_img = img.astype(np.float)
        stddev = np.random.uniform(minval, maxval)
        noise = np.random.randn(*img.shape) * stddev
        noise_img += noise
        noise_img = np.clip(noise_img, 0, 255).astype(np.uint8)
        return noise_img

In [5]:
class Rescale(object):
    
    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, sample):
        image = sample
        
        new_h, new_w = self.output_size

        img = transform.resize(image, (new_h, new_w))

        return img


In [6]:
num_epochs = 200
batch_size = 1
learning_rate = 1e-3

In [7]:
train_dateset = ImageNetDataset('/home/turing/Documents/BE/','data/', 
                               transform=transforms.Compose([
                                   addGaussian((0,50)),
                                   Rescale((256,256)),
                                   transforms.ToTensor()
                               ]))

In [8]:
test_dataset = ImageNetDataset('/home/turing/Documents/BE/','data/', 
                               transform=transforms.Compose([
                                   addGaussian((0,50)),
                                   Rescale((256,256)),
                                   transforms.ToTensor()
                               ]))

In [9]:
def getTrainLoader(batch_size):
    train_loader = DataLoader(train_dateset, batch_size=batch_size,shuffle=True, num_workers=2)
    return(train_loader)
    

In [10]:
def getTestLoader(batch_size):
    test_loader = DataLoader(test_dataset, batch_size=batch_size,shuffle=True, num_workers=2)
    
    return(test_loader)
    

In [11]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,48,3)

        self.conv2 = nn.Conv2d(48,48,3)


        self.pool = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(48,48,3)

        self.conv4 = nn.Conv2d(48,48,3)


        self.conv5 = nn.Conv2d(48,48,3)

        self.conv6 = nn.Conv2d(48,48,3)


        self.upscale = nn.Upsample(scale_factor=2)

        self.decov1 = nn.Conv2d(48,96,3)


        self.decov2 = nn.Conv2d(96,96,3)

        self.decov3 = nn.Conv2d(96,96,3)


        self.decov4 = nn.Conv2d(96,96,3)


        self.decov5 = nn.Conv2d(96,96,3)

        self.decov6 = nn.Conv2d(96,96,3,2)


        self.decov7 = nn.Conv2d(96,96,3)

        self.decov8 = nn.Conv2d(96,96,3)


        self.decov9 = nn.Conv2d(96,64,3)

        self.decov10 = nn.Conv2d(64,32,3)

        self.conv7 = nn.Conv2d(96,3,3)
        
    def forward(self, x):
        
        skips = [x]
        
        x = F.leaky_relu(self.conv1(x))
        
        x = F.leaky_relu(self.conv2(x))
        
        x = self.pool(x)
        
        skips.append(x)
        
        x = self.pool(F.leaky_relu(self.conv3(x)))
        
        skips.append(x)
        
        x = self.pool(F.leaky_relu(self.conv4(x)))
        
        skips.append(x)
        
        x = self.pool(F.leaky_relu(self.conv5(x)))
        
        
        x = F.leaky_relu(self.conv6(x))
        
        x = self.upscale(x)
        
        
        
        x = torch.cat((x, skips.pop()),1)
        print(x.shape)
        x = F.leaky_relu(self.decov1(x))
        
        x = self.upscale(F.leaky_relu_(self.decov2(x)))
        
        x = self.upscale(F.leaky_relu(self.decov3(x)))
        
        x = self.upscale(F.leaky_relu(self.decov4(x)))
        
        x = self.upscale(F.leaky_relu(self.decov5(x)))
        
        x = self.upscale(F.leaky_relu(self.decov6(x)))
        
        x = self.upscale(F.leaky_relu(self.decov7(x)))
        
        x = self.upscale(F.leaky_relu(self.decov8(x)))
        
        x = self.upscale(F.leaky_relu(self.decov9(x)))
        
        x = self.upscale(F.leaky_relu(self.decov10(x)))
        
        x = self.conv7(x)
        
        return x

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(20, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        print(x.shape)
        x = x.view(-1, 16 * 5 * 5)
        print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [12]:
def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.MSELoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [13]:
import time

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
print(device)

cuda:0


In [16]:
def trainNet(net, batch_size, n_epochs, learning_rate):
    
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    train_loader = getTrainLoader(batch_size)
    
    test_loader = getTestLoader(batch_size)
    
    n_batches = len(train_loader)
    
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    training_start_time = time.time()
    
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for inputs, labels in zip(train_loader, test_loader):
            
            
            print(inputs[0].shape)
            optimizer.zero_grad()
            inputs = inputs[0].float()
            labels = labels[0].float()
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = net(inputs)
            print(outputs.shape)
            
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            running_loss += loss_size.data[0]
            
            total_train_loss += loss_size.data[0]
            
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()

In [17]:
net = Net().to(device)
# net = Net()
trainNet(net, batch_size=batch_size, n_epochs=num_epochs, learning_rate=learning_rate)

===== HYPERPARAMETERS =====
batch_size= 1
epochs= 200
learning_rate= 0.001


/home/turing/anaconda3/envs/pytorch/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/turing/anaconda3/envs/pytorch/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/turing/anaconda3/envs/pytorch/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/turing/anaconda3/envs/pytorch/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', wi

torch.Size([1, 3, 256, 256])


/home/turing/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 30 and 24 in dimension 2 at /opt/conda/conda-bld/pytorch_1532579245307/work/aten/src/THC/generic/THCTensorMath.cu:87